In [ ]:
from pyspark.sql.types import *
from delta.tables import *
    
DeltaTable.createIfNotExists(spark) \
    .tableName("factCrimes") \
    .addColumn("CrimeID", LongType()) \
    .addColumn("ParkID", LongType()) \
    .addColumn("DateID", LongType()) \
    .addColumn("Count", LongType()) \
    .execute()

StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 17, Finished, Available, Finished)

In [ ]:
# We read the silver table 
from pyspark.sql.functions import lit
df_delta= spark.sql("SELECT * FROM Lakehouse_Projet.`nyc_park_crime_delta`")

# We don't need Total line and column as it could be calculated later
df_delta = df_delta.drop("TOTAL")
df_delta = df_delta[df_delta["PARK"] != "TOTAL"]

# We need to melt the dataframe to have crime cateogries in lines instead of columns as it is initially 
ids = ["PARK",
        "BOROUGH",
        "SIZE_ACRES",
        "CATEGORY",
        "YEAR",
        "QUARTER"]


df_melted = df_delta.melt(
        ids=ids, values=[col for col in df_delta.columns if col not in ids], 
        variableColumnName="CRIME_CATEGORY", 
        valueColumnName="COUNT")

StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 5, Finished, Available, Finished)

In [ ]:
df_melted.head(5)

StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 6, Finished, Available, Finished)

[Row(PARK='WAKEFIELD PLAYGROUND', BOROUGH='BRONX        ', SIZE_ACRES=0.877, CATEGORY='BASKETBALL & PLAYGROUND LESS THAN ONE ACRE', YEAR=2024, QUARTER='Q3', CRIME_CATEGORY='MURDER', COUNT=0),
 Row(PARK='WAKEFIELD PLAYGROUND', BOROUGH='BRONX        ', SIZE_ACRES=0.877, CATEGORY='BASKETBALL & PLAYGROUND LESS THAN ONE ACRE', YEAR=2024, QUARTER='Q3', CRIME_CATEGORY='RAPE', COUNT=0),
 Row(PARK='WAKEFIELD PLAYGROUND', BOROUGH='BRONX        ', SIZE_ACRES=0.877, CATEGORY='BASKETBALL & PLAYGROUND LESS THAN ONE ACRE', YEAR=2024, QUARTER='Q3', CRIME_CATEGORY='ROBBERY', COUNT=0),
 Row(PARK='WAKEFIELD PLAYGROUND', BOROUGH='BRONX        ', SIZE_ACRES=0.877, CATEGORY='BASKETBALL & PLAYGROUND LESS THAN ONE ACRE', YEAR=2024, QUARTER='Q3', CRIME_CATEGORY='FELONY_ASSAULT', COUNT=0),
 Row(PARK='WAKEFIELD PLAYGROUND', BOROUGH='BRONX        ', SIZE_ACRES=0.877, CATEGORY='BASKETBALL & PLAYGROUND LESS THAN ONE ACRE', YEAR=2024, QUARTER='Q3', CRIME_CATEGORY='BURGLARY', COUNT=0)]

In [ ]:
from pyspark.sql.functions import *
    

df_dimpark = spark.read.table("dimpark").filter("IsCurrent = true")  # assumes SCD with StartDate/EndDate
df_crime = spark.read.table("dimcrimecategory") # Read the current data about parks
df_date = spark.read.table("dimdate")

# Join to dimpark (SCD 2 logic: match by name + date range)
df_joined = df_melted.join(
    df_dimpark,
    (df_melted["PARK"] == df_dimpark.ParkName) &
    (df_melted["BOROUGH"] == df_dimpark.Borough) &
    (df_melted["SIZE_ACRES"] == df_dimpark.Size ) &
    (df_melted["CATEGORY"] == df_dimpark.Category ),
    "left"
    ).join(
        df_crime, 
        (df_melted["CRIME_CATEGORY"] == df_crime.crime_category),
        "left"
    ).join(
        df_date,
        (df_melted["YEAR"] == df_date.Year) &
        (df_melted["QUARTER"] == df_date.Quarter),
        "left"

    ) \
    .select(col("CrimeID"), col("DateID"), col("ParkID"), col("Count")
    )



StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 7, Finished, Available, Finished)

In [1]:
df_joined.head(10)

StatementMeta(, , -1, Waiting, , Waiting)

NameError: name 'df_joined' is not defined

In [ ]:
display(df_dimpark.limit(500))

StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c83654b3-d829-41a9-8f68-9b992fc97708)

In [ ]:
from delta.tables import DeltaTable

fact_table = DeltaTable.forName(spark, "factCrimes")


fact_table.alias("target").merge(
    df_joined.alias("source"),
    "target.ParkID = source.ParkID AND target.DateID = source.DateID AND target.CrimeID = source.CrimeID" 
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()


StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 19, Finished, Available, Finished)

In [ ]:
%%sql
SELECT *
FROM factCrimes
ORDER BY DateID
LIMIT 10;

StatementMeta(, fd1c9d5e-05c1-46bc-9c26-8cff36a95808, 20, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 4 fields>